# Dirichlet boundary conditions

This section shows how to solve the Dirichlet problem 
$$
-\Delta u  = f \quad \text{ in } \Omega 
$$
with a *nonzero Dirichlet boundary condition*
$
u|_{\Gamma_D} = g
$
on a boundary part $\Gamma_D$. 

#### Extension technique

We use the technique of reducing a problem with essential non-homogeneous boundary conditions to one with homogeneous boundary condition by extension. The solution $u$ in $H^1$ satisfies 
$$
u|_{\Gamma_D} = g$$
and 
$$ 
\int_\Omega \nabla u \cdot \nabla v_0 = \int_\Omega f v_0
$$
for all $v_0$ in $\in H_{0,D}^1 = \{ v \in H^1: v|_{\Gamma_D} = 0\}$. Split the solution 
$$
u = u_0 + u_D
$$
where $u_D$ is an extension of $g$ into $\Omega$.   Then we only need to find $u_0$ in $H^1_{0,D}$ satisfying the homogeneous Dirichlet problem 
$$
\int_\Omega \nabla u_0 \cdot \nabla v_0 = \int_\Omega f v_0 - \int_\Omega \nabla u_D \cdot \nabla v_0 
$$
for all $v_0$ in $H_{0,D}^1$.

#### Issues considered

 * How to set homogeneous Dirichlet conditions on spaces?  
 * How to define an extension $u_D$ in the finite element space?
 * How to solve for $u_0$?

### Finite element spaces with Dirichlet conditions

In [1]:
import netgen.gui
%gui tk
from ngsolve import *
from netgen.geom2d import unit_square

mesh = Mesh(unit_square.GenerateMesh(maxh=0.2))
mesh.GetBoundaries()

['bottom', 'right', 'top', 'left']

The `unit_square` has its boundaries marked as `left`, `right`, `top` and `bottom`. Suppose we want non-homogeneous Dirichlet boundary conditions on 
$$
\Gamma_D = \Gamma_{left} \cup \Gamma_{right}.
$$
Then, we set the space as follows:

In [2]:
fes = H1(mesh, order=2, dirichlet="left|right")

Compare this space with the one without the `dirichlet` flag:

In [3]:
fs2 = H1(mesh, order=2)
fes.ndof, fs2.ndof    # total number of unknowns

(125, 125)

Thus, the `dirichlet` flag did not change `ndof`. In NGSolve the unknowns are split into two groups: 
 * dirichlet dofs (or constrained dofs), 
 * free dofs.
 
The facility `FreeDofs` gives a `BitArray` such that FreeDofs[dof] is True iff dof is a free degree of freedom.

In [4]:
print("free dofs of fs2 without \"dirichlet\" flag:\n",
      fs2.FreeDofs())
print("free dofs of fes:\n", fes.FreeDofs())

free dofs of fs2 without "dirichlet" flag:
 0: 11111111111111111111111111111111111111111111111111
50: 11111111111111111111111111111111111111111111111111
100: 1111111111111111111111111
free dofs of fes:
 0: 00001111000011110000111111111111111111010101101111
50: 11111110101110111111111111111011010111111111111111
100: 1111111111111111111111111


* The space `fs2` without `dirichlet` flag has only free dofs (no dirichlet dofs).

* The other space `fes` has a few dofs that are marked as *not* free. These are the dofs that are located on boundary regions we marked as `dirichlet`.

### Forms and assembly

In NGSolve, bilinear and linear forms are defined independently of the dirichlet flags. The idea is to set up matrices and vectors with respect to all unknowns (free or constrained) and remove the constrained ones later.

Consider the Dirichlet problem with $f=1$ and $g=1$. Then we need to implement the forms in 
$$
\int_\Omega \nabla u_0 \cdot \nabla v_0 = \int_\Omega f v_0 - \int_\Omega \nabla u_D \cdot \nabla v_0.
$$

In [5]:
u = fes.TrialFunction()
v = fes.TestFunction()

a = BilinearForm(fes, symmetric=True)
a += SymbolicBFI (grad(u)*grad(v))
a.Assemble()

b = LinearForm(fes)
b += SymbolicLFI(1*v)
b.Assemble()

To complete the implementation we need $u_D$, an extension of $g=1$ from the `left` and `right` boundary parts.

### Extending the boundary values

In [6]:
g = CoefficientFunction(1)
uD = GridFunction(fes)
uD.Set(g)

* The `Set` method interpolates `g` to obtain the grid function `uD`.

* Whenever a grid function calls `Set`, it sets itself to zero everywhere before interpolation (so any of its previous values are lost).

* `Set` currently implements an Oswald-type interpolation obtained by projecting `g` in $L^2$ on each element followed by averaging interface dofs.  

In [7]:
Draw(uD, mesh, 'uD')

We see that this extension happens to be constant up to round off. Hence the last term in 
$$
\int_\Omega \nabla u_0 \cdot \nabla v_0 = \int_\Omega f v_0 - \int_\Omega \nabla u_D \cdot \nabla v_0
$$
is zero! So we may proceed to solve for $u_0$ rightaway.

### Inverting after removing Dirichlet dofs

In [9]:
u0 = GridFunction(fes)
u0.vec.data = a.mat.Inverse(freedofs=fes.FreeDofs()) * b.vec

When `FreeDofs()` are passed to `a.mat.Inverse`, the inverse is computed after removing the rows and columns corresponding to the Dirichlet degrees of freedom. 

The final solution is obtained by adding back in the extension $u_D$.

In [10]:
u = GridFunction(fes)
u.vec.data = u0.vec + uD.vec
Draw(u)

### Extending using `BND`

By some fortuitous coincidence we obtained $u_D \equiv 1$ above which allowed us to omit the last term in 
$$
\int_\Omega \nabla u_0 \cdot \nabla v_0 = \int_\Omega f v_0 - \int_\Omega \nabla u_D \cdot \nabla v_0.
$$
However this coincidence will not arise in general Dirichlet problems. 

We now create another extension $u_D$ by a better and more general way to extend boundary values. Note that the final solution $u = u_D + u_0$ must be independent of the extension!

In [11]:
g = CoefficientFunction(1)
uD = GridFunction(fes)
uD.Set(g, BND)
Draw(uD, mesh, 'uD')

The keyword `BND` tells `Set` that `g` need only be interpolated on those parts of the boundary that are marked `dirichlet`.

Now, we must add the term $-\int_\Omega \nabla u_D \cdot \nabla v_0$ to the right hand side. Since the matrix $A=$`a.mat` was assembled without boundary conditions, this amounts to adding 
$$
-A u_D
$$
to the right hand side vector `b`.

In [12]:
b.vec.data -= a.mat * uD.vec

Now we are ready to solve again.

In [13]:
u2 = GridFunction(fes)
u2.vec.data = a.mat.Inverse(freedofs=fes.FreeDofs()) * b.vec
u2.vec.data +=  uD.vec
Draw(u2)

In [14]:
sqrt(Integrate( (u2-u)*(u2-u), mesh))  

8.017366516892621e-16

Thus we see that the computed solution is indeed independent of the method of extension used. 

We now want to solve 
$$
  A (u_0 + u_D) = f \quad \Rightarrow \quad A u_0 = f - A u_D
$$
Actually, we want to solve only for the equations of $A u_0 = f - A u_D$ corresponding to free dofs. If we think of free dofs and dirichlet dofs numbered consecutively (which is typically not true), we have
$$
  \left( \begin{array}{cc} A_{FF} & A_{FD} \\ A_{DF} & A_{DD} \end{array} \right) \left( \begin{array}{c} u_{F} \\ u_{D} \end{array} \right) = \left( \begin{array}{c} {f}_F \\ {f}_D \end{array} \right) \quad \begin{array}{l} \leftarrow \text{ solved for } u_{F} \\ \leftarrow \text{ replaced with } u_{D} = g_D\end{array}
$$
which becomes
$$
  \left( \begin{array}{cc} A_{FF} & A_{FD} \\ A_{DF} & A_{DD} \end{array} \right) \left( \begin{array}{c} u_{0,F} \\ u_{0,D} \end{array} \right) = \left( \begin{array}{c} {f}_F \\ {f}_D \end{array} \right)
%  
 - \left( \begin{array}{cc} A_{FF} & A_{FD} \\ A_{DF} & A_{DD} \end{array} \right) \left( \begin{array}{c} u_{D,F} \\ u_{D,D} \end{array} \right)
% \begin{array}{l} \leftarrow \text{ solved for } u_{0,F} \\ \leftarrow \text{ replaced with } u_{0,D}  
%  \quad \begin{array}{l} \leftarrow \text{ solved for } u_{0,F} \\ \leftarrow \text{ replaced with } u_{0,D} = g_D\end{array}
$$
and with $u_{0,D} = 0$ and the second row replaced with the prescribed boundary data:
$$
  \left( \begin{array}{cc} A_{FF} &  \\  &  \end{array} \right) \left( \begin{array}{c} u_{0,F} \\ {}  \end{array} \right) = \left( \begin{array}{c} {f}_F \\ {} \end{array} \right)
%  
 - \left( \begin{array}{cc} A_{FF} & A_{FD} \\  &  \end{array} \right) \left( \begin{array}{c} u_{D,F} \\ u_{D,D} \end{array} \right) =  \tilde{f}
$$
Hence, we need to
* set up the right hand side (from a given $f$ and a given $u_D$)
* solve a linear system which involves $A_{FF}$ only
* add solution: $u = u_0 + u_D$


Setting up the right hand side (from a given $f$ and a given $u_D$) is easy:

In [15]:
f.vec.data -= a.mat * gfu.vec

Note that this affects all entries of $\tilde{f}$, although we are only interested in the parts corresponding to free dofs, $\tilde{f}_F$.

To solve for the $A_{FF}$ block we can use sparse direct solvers on $A$ with the freedofs-argument which prescribes the subblock that is to be inverted, i.e. 
$$
 A.Inverse(freedofs=...) = \left( \begin{array}{cc} A_{FF}^{-1} & 0 \\ 0 & 0 \end{array} \right)
$$
With
$$
  u = u_0 + \left( \begin{array}{cc} A_{FF}^{-1} & 0 \\ 0 & 0 \end{array} \right) \cdot \tilde{f}
$$
we simply have

In [16]:
gfu.vec.data += a.mat.Inverse(freedofs=fes.FreeDofs()) * f.vec
Redraw()

## Using BVP

Using BVP homogenization is taken care of automatically. You provide $A$, $f$, $u$ and a preconditioner and the problem is solved for. Here, the preconditioner only acts on the free dofs:

In [9]:
c = Preconditioner(a,"local")     #<- Jacobi preconditioner
#c = Preconditioner(a,"direct")   #<- sparse direct solver
c.Update()
BVP(bf=a,lf=f,gf=gfu,pre=c).Do()
Redraw()